<img src ="https://edunet.kea.su/repo/EduNet-content/L06/out/convolution_filter_forward_pass.png" width="200">

## Основные параметры свёртки

Как уже упоминалось ранее, по сути операция свёртки представляет из себя вычисление взвешенной суммы со свободным членом, что довольно сильно напоминает линейный слой с тем лишь отличием, что последний применяется ко всем данным, а не к их части. Тем не менее, подобно линейному слою, операцию свёртки можно встроить в нейросеть и, путём градиентного спуска, подбирать параметры свёртки. 

<img src ="https://edunet.kea.su/repo/EduNet-content/L06/out/img_and_convolution_filter.png" width="300">

В свёрточном слое указаны необходимые для создания ядра свёртки параметры: количество каналов во входном представлении `in_channels` и размер ядра свёртки `kernel_size`. Также в нём необходимо указывать количество используемых фильтров `out_channels`. 


In [ ]:
conv = torch.nn.Conv2d(
    in_channels=3,  # Number of input channels (3 for RGB images)
    out_channels=5,  # Number of filters/output channels
    kernel_size=3,
)

img = torch.randn(
    (1, 3, 100, 100)
)  # 1-batch size, 3-num of channels, (100,100)-img size
print(f"img shape: {img.shape}")

out = conv(img)
print(f"out shape: {out.shape}")  # [1, 5, 98, 98]

К примеру, на GIF ниже можно увидеть, как фильтр размером $3\times3$ применяется к одноканальному изображению размером $5\times5$. Шаблон имеет форму x-образного креста. В правой части можно увидеть, насколько фрагмент изображения под фильтром совпадает с шаблоном внутри фильтра.

<img src ="https://edunet.kea.su/repo/EduNet-content/L06/out/convolution_with_filter.gif" width="300">

Результатом свертки входного тензора с одним фильтром будет карта признаков с глубиной 1, вне зависимости от количества каналов входного тензора.

Во время прямого прохода фильтр перемещается по ширине и высоте входного тензора и вычисляются скалярные произведения между значениями фильтра и соответствующими значениями входного тензора. Так формируется двумерная карта признаков, которая содержит результат применения данного фильтра к каждой из областей входного тензора.



In [ ]:
img = torch.randn((3, 8, 8))  # 3-num of channels, (8,8)-img size
kernel = torch.randn((3, 3, 3))  # 3-num of filters, (3,3)-kernel size

result = torch.zeros(6, 6)  # 8 - 3 + 1 = 6

for i in range(result.shape[0]):
    for j in range(result.shape[1]):
        segment = img[:, i : i + kernel.shape[0], j : j + kernel.shape[1]]
        result[i, j] = torch.sum(segment * kernel)

print(f"img shape: {img.shape}")
print(f"kernel shape: {kernel.shape}")
print(f"result shape: {result.shape}")

Реализуем операцию свертки с помощью линейного слоя:

In [ ]:
import torch
import torch.nn as nn

local_linear = nn.Linear(9, 1, bias=False)  # 9 = 3 * 3 (weights shape: (3,3))

# fmt: off
img = torch.Tensor([[1, 1, 1, 0, 0],
                    [0, 1, 1, 1, 0],
                    [0, 0, 1, 1, 1],
                    [0, 0, 1, 1, 0],
                    [0, 1, 1, 0, 0]])
# kernel weights
weights = torch.Tensor([[1, 0, 1],
                       [0, 1, 0],
                       [1, 0, 1]])
# fmt: on

local_linear.weight = nn.Parameter(weights.reshape(-1))  # set weights

result = torch.zeros((3, 3))  # img - kernel + 1 (5 - 3 + 1 = 3)

for i in range(0, result.shape[0]):
    for j in range(0, result.shape[1]):
        segment = img[i : i + weights.shape[0], j : j + weights.shape[1]].reshape(-1)
        result[i, j] = local_linear(segment)

print(f"img shape: {img.shape}")
print(f"weights shape: {weights.shape}")
print(f"result shape: {result.shape}")
print(f"result:\n{result}")

```
"VALID" = без паддинга:

   inputs:         1  2  3  4  5  6  7  8  9  10 11 (12 13)
                  |________________|                dropped
                                 |_________________|
"SAME" = с паддингом нулями:

               pad|                                      |pad
   inputs:      0 |1  2  3  4  5  6  7  8  9  10 11 12 13|0  0
               |________________|
                              |_________________|
                                             |________________|
```

Ниже можно увидеть пример обработки RGB изображения с same padding (с использованием 0):